In [1]:
from sklearn import datasets
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import jieba


In [2]:
df = pd.read_csv(r'movies.csv')

In [3]:
tags = []
for i in df.分類:
    counter = 0
    tag = ""
    stop = ["[", "]", "'", ",", " "]
    for j in i:
        if counter == 2:
            break
        if j not in stop:
            tag = tag + j
            counter += 1
    tags.append(tag)

In [4]:
info = {}
infos = []
counter = 0
for i in range(len(df)):
    info = {}
    temp = tags[counter]
    if temp == "":
        counter += 1
        continue
    info['名稱'] = df.名稱[counter]
    info['分類'] = temp
    info['劇情介紹'] = df.劇情介紹[counter]
    counter += 1
    infos.append(dict(list(info.items())))

In [5]:
tag_list = []
for i in infos:
    temp = i['分類']
    if temp not in tag_list:
        tag_list.append(temp)

In [6]:
tag_code = []
test_code = []
for cut, i in enumerate(infos):
    if cut < len(infos) - 500:
        for counter, j in enumerate(tag_list):
            if i['分類'] == j:
                tag_code.append(counter)
    else:
        for counter, j in enumerate(tag_list):
            if i['分類'] == j:
                test_code.append(counter)

In [7]:
#用標題來預測
title = []
test = []
for counter, i in enumerate(infos):
    words = i['名稱'].split()
    cut_words = [list(jieba.cut(j)) for j in words]
    if counter < len(infos) - 500:
        title.append([" ".join(j) for j in cut_words])
    else:
        test.append([" ".join(j) for j in cut_words])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\輝夜\AppData\Local\Temp\jieba.cache
Loading model cost 0.787 seconds.
Prefix dict has been built successfully.


In [8]:
arr = []
for i in title:
    arr.append(i[0])

In [9]:
test_arr = []
for i in test:
    test_arr.append(i[0])

In [10]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
response = vectorizer.fit(arr).transform(arr)
test_data = vectorizer.fit(arr).transform(test_arr)

In [11]:
knn = KNeighborsClassifier(n_neighbors = 4)
knn.fit(response, tag_code)
result = knn.predict(test_data)

In [12]:
temp = len(result)
success = 0
if temp == len(test_code):
    for i in range(temp):
        if result[i] == test_code[i]:
            success += 1
print(success/temp)

0.422


In [13]:
#用劇情介紹來預測
intro = []
test2 = []
for counter, i in enumerate(infos):
    words = i['劇情介紹'].split()
    cut_words = [list(jieba.cut(j)) for j in words]
    if counter < len(infos) - 500:
        intro.append([" ".join(j) for j in cut_words])
    else:
        test2.append([" ".join(j) for j in cut_words])

In [14]:
arr2 = []
for i in intro:
    arr2.append(i[0])

In [15]:
test_arr2 = []
for i in test2:
    test_arr2.append(i[0])

In [16]:
#vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
response2 = vectorizer.fit(arr2).transform(arr2)
test_data2 = vectorizer.fit(arr2).transform(test_arr2)

In [17]:
knn2 = KNeighborsClassifier(n_neighbors = 3)
knn2.fit(response2, tag_code)
result2 = knn2.predict(test_data2)

In [18]:
temp = len(result2)
success = 0
if temp == len(test_code):
    for i in range(temp):
        if result2[i] == test_code[i]:
            success += 1
print(success/temp)

0.342
